## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/vsirazenzadnji.csv",delimiter=' ',header=None)

In [ ]:
X= dataset.iloc[:,:-1].values
y= dataset.iloc[:,-1].values

In [ ]:
index = pd.Index(y)
index.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66)

In [ ]:

scaler = preprocessing.StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
y_train

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = tf.keras.layers.Dense(64,activation="relu")(inputs)
x = tf.keras.layers.Dense(32,activation="relu")(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
outputs = tf.keras.layers.Dense(5, activation='softmax')(x)


convnn = tf.keras.Model(inputs=inputs, outputs=outputs)
print(convnn.summary())

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

expand_dims = tf.expand_dims(inputs, axis=2)

gru = tf.keras.layers.GRU(256, return_sequences=True)(expand_dims)

flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(5, activation='softmax')(flatten)


recnn = tf.keras.Model(inputs=inputs, outputs=outputs)
print(recnn.summary())

In [ ]:
convnn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = convnn.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)




In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Rezultat

In [ ]:
model_acc = convnn.evaluate(X_test, y_test, verbose =0)[1]
print("Natančnost modela: {:.3f}%".format(model_acc*100))

In [ ]:
y_pred = np.array(list(map(lambda x: np.argmax(x) , model.predict(X_test))))
label_dict = {"Gnus":0, "strah":1, "žalost":2 , "nevtralno":3, "veselje":4}
cm = confusion_matrix(y_test,y_pred)
clr = classification_report(y_test, y_pred, target_names=label_dict.keys())

In [ ]:
plt.figure(figsize= (8,8))
sns.heatmap(cm, annot= True, fmt="g",cbar =False, cmap = "Blues")
plt.xticks(np.arange(5)+0.5)
plt.yticks(np.arange(5)+0.5)
plt.xlabel("napovedano")
plt.ylabel("dejansko")
plt.title("tabela napačnih klasifikacij")
plt.show()


print("Classification Report:\n----------------------\n", clr)

In [ ]:
testset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/eeg model/zadnji.csv",delimiter=" ", header=None)
new_X = testset.iloc[:,:-1].values
new_y = testset.iloc[:,-1].values

In [ ]:
new_X = scaler.fit_transform(new_X)

In [ ]:
model_acc = convnn.evaluate(new_X, new_y, verbose =0)[1]
print("Natančnost modela: {:.3f}%".format(model_acc*100))

In [ ]:
testset.head()


In [ ]:
ynovi_pred = np.array(list(map(lambda x: np.argmax(x) , model.predict(new_X))))
nov_cm = confusion_matrix(new_y,ynovi_pred)
nov_clr = classification_report(new_y, ynovi_pred, target_names=label_dict.keys())

In [ ]:
plt.figure(figsize= (8,8))
sns.heatmap(cm, annot= True, fmt="g",cbar =False, cmap = "Blues")
plt.xticks(np.arange(5)+0.5, label_dict.keys())
plt.yticks(np.arange(5)+0.5, label_dict.keys())
plt.xlabel("napovedano")
plt.ylabel("dejansko")
plt.title("tabela napačnih klasifikacij za posameznika (16 participant)")
plt.show()

print("Classification Report za posameznika:\n----------------------\n", nov_clr)

##CV

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 3
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(256, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(no_classes, activation='softmax'))

  # Compile the model
  model.compile(loss=loss_function,
                optimizer=optimizer,
                metrics=['accuracy'])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1
